In [12]:
import pandas as pd
import numpy as np
import sktime
import seaborn as snsc
import matplotlib.pyplot as plt
from convertcsv.import_preprocess_v4 import readcsvs
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sktime.classification.kernel_based import RocketClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.impute import KNNImputer
from convertcsv.get_all_metrics_with_tags import get_all_metrics_with_tags
from visualization import graphs
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sktime.datatypes import convert_to
from sktime.datatypes import MTYPE_REGISTER
from collections import Counter
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.compose import ClassifierPipeline
from storage import store_csv


In [13]:
#What should we do here?

#Set up many different pipelines to compare.

#The things to vary, in order of significance/importance
"""
NaN treshold
"""
file_list, y = get_all_metrics_with_tags(r"F:\Master\Kubernetes\sockshop\microservices-demo\query\automated\generated_csvs_4")
initial_readings = readcsvs(file_list, reduce_NaNs_treshold=False, remove_unique_cols=True, remove_monotonic_increasing=True)


In [14]:
#Prior testing revealed optimal KNN imputation
imputer = KNNImputer(n_neighbors=1)
complete_value_set = imputer.fit_transform(initial_readings)
imputed_df = pd.DataFrame(complete_value_set, index=initial_readings.index, columns=initial_readings.columns)

In [15]:
#Find the length of each individual time series
def trimming(df:pd.DataFrame, y, min_percent=90):
    instances = df.index.get_level_values(0).unique()
    timeSeriesLengths = []
    for instance in instances:
        #This line of code gets every second level index value that corresponds to the current first level index value. 
        second_level_values = df.index.get_level_values(1)[df.index.get_level_values(0)==instance]
        length = len(second_level_values)
        timeSeriesLengths.append(length)
    #Find the most common value.
    counts = Counter(timeSeriesLengths)
    most_common_value, most_common_value_count = counts.most_common(1)[0]
    percentage = (most_common_value_count / len(timeSeriesLengths)) *100

    if percentage >= min_percent:
        indicies_to_drop = []
        for indice in range(len(timeSeriesLengths)):
            if timeSeriesLengths[indice] != most_common_value:
                indicies_to_drop.append(indice)
        filtered_df:pd.DataFrame = df[~df.index.get_level_values(0).isin(indicies_to_drop)]
        #Reset the index
        current_indice = 0
        desired_indice = 0
        newindex_tuples = []
        for index_tuple in filtered_df.index:
        
            #nevermind just set it equals, we only care about change anyway
            if(index_tuple[0] != current_indice):
                #this way, when the index being looked at changes, we know.
                current_indice = index_tuple[0]
                #we logged that there is a change so we know its time for next indice
                desired_indice += 1
 
            newindex_tuples.append((desired_indice, index_tuple[1]))
        
        newindex = pd.MultiIndex.from_tuples(newindex_tuples, names=filtered_df.index.names)
        newdf = pd.DataFrame(filtered_df.values, columns=filtered_df.columns, index=newindex)
            
        y = np.array(y)

        #Now rebuild the multiindex to be monotonically increasing.
        
        
        #This still keeps the fucky index in the thing

        return newdf, np.delete(y, indicies_to_drop)

    return ValueError(percentage)
    #To properly preprocess, one should check if the most common value equals the max value. If it does, great.
    #Imagine we find a perfect thing for it. Now we have a couple of datasets consisting of multivariate data.
    #It's important to preserve the general shape of the dataset. So just adding mean to the front and back probably isn't that good. 
    #Can decide to simply purge the ones that aren't of the correct length. To do this, you would have to decide that the most common value is the highest one, and that there are very few time series that differ. Incurs data loss but saves a lot of time.
    #Instead of getting bogged down in a custom optimal solution, get the alright one. For now, get a determiner that purging the ones that are too small is ok. 
    #If ok, proceed. Can then later compare with other algorithms to see if the purging was helpful.
    #Write in the thesis that shit happens and time constraints wcyd

    #For now, determine if purging is ait
    #The most common value should be at least 90% to only tolerate 10% data loss perhaps.

trimmed_df , trimmed_y = trimming(imputed_df, y)


In [16]:
store_csv.store_csv(trimmed_df, trimmed_y,)

"'f:\\\\Master\\\\Kubernetes\\\\sockshop\\\\microservices-demo\\\\analysis\\\\storage'"